In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from folium.plugins import MarkerCluster

# ======================
# 1. Cargar datos
# ======================
# Ejemplo de DataFrame (reemplace con su archivo real)
df = pd.read_excel('Muestra de farmacias.xlsx',sheet_name='base',dtype = object)

# ======================
# 2. Clusterización
# ======================
X = df[['latitud', 'longuitud']].values
k = 100  # número de clusters (ajústelo según su caso)
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X)

# ======================
# 3. Crear el mapa base centrado en Ecuador
# ======================
m = folium.Map(location=[-1.8312, -78.1834], zoom_start=6, tiles="CartoDB positron")

# Colores para los clusters
colors = ["red", "blue", "green", "purple", "orange", "darkred", 
          "lightblue", "lightgreen", "beige", "pink"]

# ======================
# 4. Añadir puntos de tiendas
# ======================
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longuitud']],
        radius=6,
        color=colors[row['cluster'] % len(colors)],
        fill=True,
        fill_color=colors[row['cluster'] % len(colors)],
        fill_opacity=0.8,
        popup=f"Sucursal: {row['farmacia']}<br>Cluster: {row['cluster']}"
    ).add_to(m)

# ======================
# 5. (Opcional) Añadir marcadores de los centroides
# ======================
for idx, center in enumerate(kmeans.cluster_centers_):
    folium.Marker(
        location=[center[0], center[1]],
        icon=folium.Icon(color="black", icon="star"),
        popup=f"Centroide Cluster {idx}"
    ).add_to(m)

# Guardar mapa en HTML
m.save("clusters_ecuador.html")
df.to_excel("farmacias_con_clusters.xlsx", index=False)

In [ ]:
df

In [12]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import folium

# ======================
# 1. Cargar datos
# ======================
# Ejemplo de DataFrame (reemplace con su archivo real)
df = pd.read_excel('Muestra de farmacias.xlsx',sheet_name='base',dtype = object)
df['latitud'] = df['latitud'].astype(float)
df['longuitud'] = df['longuitud'].astype(float)
# ======================
# 2. Clusterización con DBSCAN usando haversine
# ======================
# Convertir lat/lon a radianes
coords = np.radians(df[['latitud', 'longuitud']].values)

# eps en radianes (10 km en radio terrestre ~6371 km)
kms_per_radian = 6371.0
eps = 2 / kms_per_radian

# DBSCAN
db = DBSCAN(eps=eps, min_samples=2, metric='haversine').fit(coords)
df['cluster'] = db.labels_

# ======================
# 3. Crear mapa en folium
# ======================
m = folium.Map(location=[-1.8312, -78.1834], zoom_start=6, tiles="CartoDB positron")

colors = ["red", "blue", "green", "purple", "orange", "darkred", 
          "lightblue", "lightgreen", "beige", "pink"]

for i, row in df.iterrows():
    cluster_id = row['cluster']
    color = "gray" if cluster_id == -1 else colors[cluster_id % len(colors)]
    
    folium.CircleMarker(
        location=[row['latitud'], row['longuitud']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=f"Sucursal: {row['farmacia']}<br>Cluster: {cluster_id}"
    ).add_to(m)

m.save("clusters_por_distancia.html")